In [292]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris, load_diabetes
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [293]:

caminho_dataset = "heart.csv"
dados = pd.read_csv(caminho_dataset) 

X = dados.to_numpy()
y = X[:, -1:]
X = X[:, :-1]

X.shape, y.shape

((303, 13), (303, 1))

In [294]:
'''
#A base de dados está em: https://www.kaggle.com/spscientist/students-performance-in-exams
caminho_dataset = "StudentsPerformance.csv"
dados = pd.read_csv(caminho_dataset) 

#transformando "math_score" em classes de avaliação
def math_score_to_class(x):
    if x <= 59:
        return 0    
    else:
        return 1

dados["math score"] = dados["math score"].apply(math_score_to_class)


X = dados.to_numpy()
y = X[:,5:6]
X = X[:,:5]

y = y.astype('int')
y = y.ravel()

label_encoder = LabelEncoder()

#transformando "Gender" em número inteiro
X[:,0:1] = label_encoder.fit_transform(X[:,0:1]).reshape(X.shape[0], 1)

#transformando "race/ethnicity" em número inteiro
X[:,1:2] = label_encoder.fit_transform(X[:,1:2]).reshape(X.shape[0], 1)

#transformando "parental level of education" em número inteiro
X[:,2:3] = label_encoder.fit_transform(X[:,2:3]).reshape(X.shape[0], 1)

#transformando "lunch" em número inteiro
X[:,3:4] = label_encoder.fit_transform(X[:,3:4]).reshape(X.shape[0], 1)

#transformando "test preparation course" em número inteiro
X[:,4:5] = label_encoder.fit_transform(X[:,4:5]).reshape(X.shape[0], 1)
'''

'\n#A base de dados está em: https://www.kaggle.com/spscientist/students-performance-in-exams\ncaminho_dataset = "StudentsPerformance.csv"\ndados = pd.read_csv(caminho_dataset) \n\n#transformando "math_score" em classes de avaliação\ndef math_score_to_class(x):\n    if x <= 59:\n        return 0    \n    else:\n        return 1\n\ndados["math score"] = dados["math score"].apply(math_score_to_class)\n\n\nX = dados.to_numpy()\ny = X[:,5:6]\nX = X[:,:5]\n\ny = y.astype(\'int\')\ny = y.ravel()\n\nlabel_encoder = LabelEncoder()\n\n#transformando "Gender" em número inteiro\nX[:,0:1] = label_encoder.fit_transform(X[:,0:1]).reshape(X.shape[0], 1)\n\n#transformando "race/ethnicity" em número inteiro\nX[:,1:2] = label_encoder.fit_transform(X[:,1:2]).reshape(X.shape[0], 1)\n\n#transformando "parental level of education" em número inteiro\nX[:,2:3] = label_encoder.fit_transform(X[:,2:3]).reshape(X.shape[0], 1)\n\n#transformando "lunch" em número inteiro\nX[:,3:4] = label_encoder.fit_transform(X[:,

# Usando apenas Validação Cruzada - KNN e Logistic Regression

In [295]:
classificador_knn = KNeighborsClassifier()

scores_cv = cross_validate(classificador_knn, X, y, scoring='accuracy', cv=10)
print(scores_cv['test_score'])
sempad = np.mean(scores_cv['test_score'])    
print(f"Acurácia: {sempad}")

[0.70967742 0.67741935 0.5483871  0.63333333 0.63333333 0.56666667
 0.76666667 0.7        0.53333333 0.76666667]
Acurácia: 0.6535483870967742


In [296]:
classificador_logreg = LogisticRegression()

scores_cv = cross_validate(classificador_logreg, X, y, scoring='accuracy', cv=10)
print(scores_cv['test_score'])
sempad = np.mean(scores_cv['test_score'])
print(f"Acurácia: {sempad}")

[0.87096774 0.80645161 0.83870968 0.9        0.9        0.86666667
 0.83333333 0.86666667 0.7        0.73333333]
Acurácia: 0.8316129032258065


# Usando Pipeline(com padronização dos dados) dentro do Grid Search - KNN e Logistic Regression

In [297]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", classificador_knn)
])

parametros = {'knn__n_neighbors': [3,5,7]}

grid_search_pipeline = GridSearchCV(pipeline, parametros, scoring='accuracy')

scores_cv_gs_pipe = cross_validate(grid_search_pipeline, X, y, scoring='accuracy', cv=10)
print(scores_cv_gs_pipe['test_score'])
compad = np.mean(scores_cv_gs_pipe['test_score'])
print(f"Acurácia: {compad}")

[0.87096774 0.80645161 0.80645161 0.86666667 0.83333333 0.76666667
 0.9        0.83333333 0.7        0.73333333]
Acurácia: 0.8117204301075269


In [298]:
pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("logreg", classificador_logreg)
])


parametros = {'logreg__C': [0.01, 0.1, 1, 10, 100, 1000, 10000] }

grid_search_pipeline = GridSearchCV(pipeline, parametros, scoring='accuracy')

scores_cv_gs_pipe = cross_validate(grid_search_pipeline, X, y, scoring='accuracy', cv=10)
print(scores_cv_gs_pipe['test_score'])
compad = np.mean(scores_cv_gs_pipe['test_score'])
print(f"Acurácia: {compad}")

[0.87096774 0.80645161 0.83870968 0.86666667 0.86666667 0.83333333
 0.86666667 0.83333333 0.7        0.73333333]
Acurácia: 0.8216129032258065


No KNeighborsClassifier, a padronização dos dados contribuiu muito para um salto na acurácia. Já no LogisticRegression, usando Pipeline + GridSearch + padronização dos dados, a acurácia diminuiu em relação ao cross_validate sozinho.  

Troquei a base de dados pois 100% de acurácia já havia sido alcançado pelo LogisticRegression.